<a href="https://colab.research.google.com/github/rahiakela/kaggle-competition-projects/blob/master/meta-kaggle/01-linear-regression/01_linear_regression_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Linear Regression from scratch

In [1]:
import numpy as np
import torch

In [2]:
# Create tensors
x = torch.tensor(3.0)
w = torch.tensor(4.0, requires_grad=True)
b = torch.tensor(5.0, requires_grad=True)

We can combine tensors with the usual arithmetic operations.

In [3]:
# Arithmetic operations
y = w * x + b
print(y)

tensor(17., grad_fn=<AddBackward0>)


What makes PyTorch special, is that we can automatically compute the derivative of `y` w.r.t. the tensors that have `requires_grad` set to `True` i.e. `w` and `b`.

In [4]:
# Compute gradients
y.backward()

In [5]:
# Display gradients
print(f"dy/dw: {w.grad}")
print(f"dy/db: {b.grad}")

dy/dw: 3.0
dy/db: 1.0


##Problem Statement

We'll create a model that predicts crop yeilds for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. Here's the training data:

<img src="https://i.imgur.com/lBguUV9.png" width="500" />

In a **linear regression** model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yeild_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.

<img src="https://i.imgur.com/mtkR2lB.png" width="540" >


**Our objective**: Find a suitable set of *weights* and *biases* using the training data, to make accurate predictions.

##Training Data

The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column per variable.

In [6]:
# Input (temp, rainfall, humidity)
inputs = np.array([
    [73, 67, 43],
    [91, 88, 64],
    [87, 134, 58],
    [102, 43, 37],
    [69, 96, 70]
], dtype="float32")

In [7]:
# Targets (apples, oranges)
targets = np.array([
    [56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]
], dtype="float32")

Before we build a model, we need to convert inputs and targets to PyTorch tensors.

In [8]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


##Linear Regression Model

The *weights* and *biases* can also be represented as matrices, initialized with random values. The first row of `w` and the first element of `b` are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [9]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

print(w)
print(b)

tensor([[-0.8192,  0.5226,  0.1578],
        [ 0.9957,  0.9839,  0.3789]], requires_grad=True)
tensor([1.1408, 1.9003], requires_grad=True)


The *model* is simply a function that performs a matrix multiplication of the input `x` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

$$
\hspace{2.5cm} X \hspace{1.1cm} \times \hspace{1.2cm} W^T \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\times
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$

In [10]:
# Define the model
def model(x):
  return x @ w.t() + b

The matrix obtained by passing the input data to the model is a set of predictions for the target variables.

In [11]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-16.8655, 156.7958],
        [-17.3243, 203.3357],
        [  9.0457, 242.3373],
        [-54.1123, 159.7845],
        [  5.8262, 191.5750]], grad_fn=<AddBackward0>)


In [12]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Because we've started with random weights and biases, the model does not a very good job of predicting the target varaibles.

##Loss Function

We can compare the predictions with the actual targets, using the following method:
* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [13]:
# MSE loss
def mse(t1, t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()

In [14]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(9260.6738, grad_fn=<DivBackward0>)


The resulting number is called the **loss**, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

##Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the `loss` w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [15]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors.

In [16]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.8192,  0.5226,  0.1578],
        [ 0.9957,  0.9839,  0.3789]], requires_grad=True)
tensor([[-7660.2314, -8173.9819, -5084.3276],
        [ 8538.5371,  8343.7988,  5249.3091]])


In [17]:
# Gradients for bias
print(b)
print(b.grad)

tensor([1.1408, 1.9003], requires_grad=True)
tensor([-90.8860,  98.7657])


A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

* If a gradient element is **postive**,
    * **increasing** the element's value slightly will **increase** the loss.
    * **decreasing** the element's value slightly will **decrease** the loss.

<img src="https://i.imgur.com/2H4INoV.png" width="400" />



* If a gradient element is **negative**,
    * **increasing** the element's value slightly will **decrease** the loss.
    * **decreasing** the element's value slightly will **increase** the loss.
    
<img src="https://i.imgur.com/h7E2uAv.png" width="400" />    

The increase or decrease is proportional to the value of the gradient.

Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


##Adjust Gradients

We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [19]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-16.8655, 156.7958],
        [-17.3243, 203.3357],
        [  9.0457, 242.3373],
        [-54.1123, 159.7845],
        [  5.8262, 191.5750]], grad_fn=<AddBackward0>)


In [20]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(9260.6738, grad_fn=<DivBackward0>)


In [21]:
# Compute gradients
loss.backward()

In [22]:
# Adjust weights & reset gradients
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

In [23]:
print(w)

tensor([[-0.7426,  0.6043,  0.2086],
        [ 0.9103,  0.9004,  0.3264]], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(6333.8018, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [25]:
# Train for 100 epochs
for i in range(100):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [26]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(101.7731, grad_fn=<DivBackward0>)


In [27]:
# Print predictions
preds

tensor([[ 57.3239,  74.9878],
        [ 80.2405,  98.2503],
        [122.9137, 130.9461],
        [ 20.6816,  61.8064],
        [ 98.9587, 100.8342]], grad_fn=<AddBackward0>)

In [28]:
# Print targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

##PyTorch Linear Regression

Let's re-implement the same model using some built-in functions and classes from PyTorch.